In [1]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace


/workspace


In [2]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [3]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


In [4]:
date: str = "2022年8月12日"

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    # "modelLinAndIp",
    # "modelLinAndLog",
    # "modelIpAndLin",
    # "modelIpAndLog",
    # "modelLogAndLin",
    # "modelLogAndIp",
    # "modelProcessDividedByProblemSize",
    # "modelProblemSizeDividedByProcess",
    # "modelLinearSumOf2elementCombination",
    # "modelLinearSumOfElementCombinations",
    # "modelLinearSumOf2elementCombinationWithSquared",
    # "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

mg_input_list_problem_size: list[int] = [32, 64, 128, 256]
mg_input_list_nit: list[int] = [4, 10, 20, 35]

mg_target_list_problem_size: list[int] = [512]
mg_target_list_nit: list[int] = [50]

In [5]:
input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=input_list_process,
    list_na=cg_input_list_na,
    list_nonzer=cg_input_list_nonzer,
    list_niter=cg_input_list_niter,
    list_shift=cg_input_list_shift,
    csvDir=csvDirPath,
)

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_nonzer=cg_target_list_nonzer,
    list_niter=cg_target_list_niter,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)

/workspace/lib/lab_lib.py:8775: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/workspace/lib/lab_lib.py:8775: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [6]:
cg_list_exp: list[str] = ["process", "na", "nonzer", "niter", "shift"]

list_res: list[str] = ["#Call"]

In [7]:
# 精度の低い関数を抽出したうえでチューニングする

targetFunctionNames: list[str] = [
    "VECSET",
    "MPI_Wait()",
    "MPI_Send()",
    "SPRNVC",
    "RANDLC",
    "MPI_Irecv()",
    "ICNVRT",
]

In [8]:
result_series_list: list[pd.DataFrame] = []

function_names: list[str] = list(set(input_rawDF_cg["Name"].tolist()))

input_rawDF_cg = input_rawDF_cg.rename(columns={"Name": "functionName"})
target_rawDF_cg = target_rawDF_cg.rename(columns={"Name": "functionName"})

for function_name in function_names:
    if (function_name in targetFunctionNames) == False:
        continue
    input_rawDF_per_function: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == function_name
    ]

    models = Models(
        inputDF=input_rawDF_per_function,
        expVarColNames=cg_list_exp,
        resVarColNames=list_res,
        targetDF=None,
        modelNames=list_modelName,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    dictCalcedMAPE = models.returnCalculatedMAPE()

    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

    dict_for_series: dict = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = function_name

    series: pd.Series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF: pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)
resultDF = addLowestMAPEsModelNameColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)

return_expect=
[[ 360695.]
 [ 360695.]
 [ 360695.]
 ...
 [4721790.]
 [4721790.]
 [4721790.]]
return_actuall=
[  19151.17730987   19080.96660054   19010.75589122 ... 4685216.83369645
 4685146.62298713 4685041.30692315]
return_expect=
[[ 1680.]
 [ 1680.]
 [ 1680.]
 ...
 [31122.]
 [31122.]
 [31122.]]
return_actuall=
[  167.66127473   167.60015787   167.53904101 ... 27191.29167183
 27191.23055497 27191.13887968]
return_expect=
[[ 1680.]
 [ 1680.]
 [ 1680.]
 ...
 [31122.]
 [31122.]
 [31122.]]
return_actuall=
[  167.66127473   167.60015787   167.53904101 ... 27191.29167183
 27191.23055497 27191.13887968]
return_expect=
[[ 1680.]
 [ 1680.]
 [ 1680.]
 ...
 [31122.]
 [31122.]
 [31122.]]
return_actuall=
[  167.66127473   167.60015787   167.53904101 ... 27191.29167183
 27191.23055497 27191.13887968]
return_expect=
[[ 14000.]
 [ 14000.]
 [ 14000.]
 ...
 [100000.]
 [100000.]
 [100000.]]
return_actuall=
[[ 14000]
 [ 14000]
 [ 14000]
 ...
 [100000]
 [100000]
 [100000]]
return_expect=
[[ 180347.]
 [ 1

In [9]:
resultDF

,modelLin,modelIp,modelLog,modelSquareRootOfProcess,modelSquareRootTimesOtherElems,modelObeyOneParameter,functionName,最低値,最適モデル
0,2.550716e+01,63.602569,43.918421,2.550716e+01,110.789118,25.507161,RANDLC,25.507157,modelSquareRootOfProcess
1,3.425747e+01,52.562966,45.284269,3.761452e+01,63.622956,34.257466,MPI_Send(),34.257466,modelObeyOneParameter
2,3.425747e+01,52.562966,45.284269,3.761452e+01,63.622956,34.257466,MPI_Irecv(),34.257466,modelObeyOneParameter
3,3.425747e+01,52.562966,45.284269,3.761452e+01,63.622956,34.257466,MPI_Wait(),34.257466,modelObeyOneParameter
4,1.056736e-13,40.738362,24.184347,3.326790e-13,79.407704,0.000000,VECSET,0.000000,modelObeyOneParameter
5,2.550709e+01,63.602611,43.918416,2.550709e+01,110.789284,25.507090,ICNVRT,25.507085,modelSquareRootOfProcess
6,1.056736e-13,40.738362,24.184347,3.326790e-13,79.407704,0.000000,SPRNVC,0.000000,modelObeyOneParameter


In [10]:
df_ICNVRT = input_rawDF_cg[input_rawDF_cg["functionName"] == "ICNVRT"]
df_Wait = input_rawDF_cg[input_rawDF_cg["functionName"] == "MPI_Wait()"]
df_Send = input_rawDF_cg[input_rawDF_cg["functionName"] == "MPI_Send()"]
df_RANDLC = input_rawDF_cg[input_rawDF_cg["functionName"] == "RANDLC"]
df_VECSET = input_rawDF_cg[input_rawDF_cg["functionName"] == "VECSET"]
df_Irecv = input_rawDF_cg[input_rawDF_cg["functionName"] == "MPI_Irecv()"]
df_SPRNVC = input_rawDF_cg[input_rawDF_cg["functionName"] == "SPRNVC"]

In [11]:
# SPRNVC のデータで Model_obeyOneParameter_ForMultipleRegression を用いて予測を行う

# df_SPRNVC

print(df_SPRNVC[cg_list_exp].head())
print(df_SPRNVC[list_res].head())

object_obeyOneParameter = Model_obeyOneParameter_ForMultipleRegression(
    inputDF=df_SPRNVC,
    explanatoryVariableColumnNames=cg_list_exp,
    responseVariableColumnNames=list_res,
    conditionDictForTest={},
    targetDF=None
)

object_obeyOneParameter.build_model()

_return_expect :np.ndarray = object_obeyOneParameter.rawResponseVariable[object_obeyOneParameter.responseVariableColumnNames].values
_return_actually :np.ndarray = object_obeyOneParameter.predict(object_obeyOneParameter.rawExplanaoryVariable)

   process     na  nonzer  niter  shift
7        2  14000      11     15     20
7        2  14000      11     15     40
7        2  14000      11     15     60
7        2  14000      11     15     80
7        2  14000      11     15    110
     #Call
7  14000.0
7  14000.0
7  14000.0
7  14000.0
7  14000.0


In [12]:
_return_actually

array([[ 14000],
       [ 14000],
       [ 14000],
       ...,
       [100000],
       [100000],
       [100000]])

In [13]:
_return_expect

array([[ 14000.],
       [ 14000.],
       [ 14000.],
       ...,
       [100000.],
       [100000.],
       [100000.]])

In [14]:
test_Model_obeyOneParameter_ForMultipleRegression()

return_expect=
[[838.76639126]
 [700.66108261]
 [856.26625017]
 [641.51598139]
 [373.3698381 ]
 [156.4944123 ]
 [546.28657979]
 [  2.41803077]
 [ 42.51089236]
 [ 56.54239562]
 [737.17921771]]
return_actuall=
[[838.76639126]
 [700.66108261]
 [856.26625017]
 [641.51598139]
 [373.3698381 ]
 [156.4944123 ]
 [546.28657979]
 [  2.41803077]
 [ 42.51089236]
 [ 56.54239562]
 [737.17921771]]


In [15]:
%reset

Nothing done.


***

---

___

# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 


***

---

___